In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]


# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)

    # 変更点：ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,  # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph


# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values",
    )
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)


# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# モデル名
MODEL_NAME = "gpt-4o-mini"

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1たす2は？
1たす2は3です。何か他に質問がありますか？
台湾観光について検索結果を教えて

[{"url": "https://newt.net/twn/tpe/mag-974247954244", "content": "photo by Shutterstock1日目　台北到着→新幹線で高雄へ→名物の牛肉麺ランチ→高雄市内観光（蓮池潭、美麗島駅）→駁二芸術特区で散策を楽しむ→愛河で夕日観賞→六合夜市で食べ歩き2日目　高雄から台鉄で台南へ→度小月担仔麺でランチ→林百貨でショッピング→神農街でノスタルジック散策3日目　台南市内観光（赤嵌樓、安平樹屋）→台南から台北へ→そのまま九份へ4日目　龍山寺や迪化街で街歩き→鼎泰豊本店で小籠包ランチ→お土産探し→帰国\n\n台湾観光で見どころ満載の旅行を満喫しよう！\n\nこの記事では、台湾のおすすめの観光地とモデルコースを紹介しました。台北市をはじめ、台南、高雄、台中など名所や見どころがいっぱいある台湾では、効率よくグルメも観光も楽しむのがおすすめです。ぜひ紹介したスポットやモデルコースを参考に、オリジナルの台湾観光のスケジュールを立ててみてくださいね！※2023年9月20日のレート、1台湾ドル＝4.63円で計算していますcover photo by shutterstock\n\nおトクなフライトつきツアーも🛫 [...] photo by pixabay台湾は、日本から4時間ほどで行ける人気の観光地。日本からの直行便も多く、費用も格安で弾丸で台湾観光を楽しめるのも魅力のひとつです。親日家が多く治安もいいので、台湾は初めての海外旅行先にもおすすめ！台湾には屋台が並ぶ夜市や、映画『千と千尋の神隠し』のモデルともいわれる九份、幻想的なランタンの世界を楽しめる十分などの観光名所や見どころスポットもいっぱいです。小籠包、かき氷、魯肉飯などの台湾グルメも欠かせません。\n\n台湾観光のベストシーズンは？\n\nphoto by pixabay台湾の気候は、1年中を通して暖かく夏のような気候が続き、とくに、3月から5月、9月下旬から11月にかけては台湾観光のベストシーズン。カラッと晴れる日が多くて台湾観光にもおすすめの時期です。1月から2月にかけては短いですが冬のよう